### Loading Data

In [1]:
import pandas as pd
import time

# Input path and filename
path = 'D:/Git_Projects/5. Master_thesis/Datasets/'
filename = path + 'Retails_ALL_CY11_CY22.xlsx'
external_file = path + 'External_indicators.xlsx'

start_time = time.time()

# Load file into pandas dataframe
data_int = pd.concat(pd.read_excel(filename, sheet_name=None), ignore_index=True)
data_ext = pd.concat(pd.read_excel(external_file, sheet_name=None), ignore_index=True)

end_time = time.time()
time_elapsed = end_time - start_time 

print('Completed in {:0.2f} minutes'.format(time_elapsed/60))

Completed in 2.69 minutes


### Pre processing data

In [2]:
df = data_int.copy()
df_ext = data_ext.copy()

In [3]:
# Remove potential extra spaces in cells using strip() function
df['RetailCountry'] = df['RetailCountry'].str.strip()
df['orderNumber'] = df['orderNumber'].str.strip()

# Sort by orderNumber and RetailDate
df.sort_values(by=['orderNumber', 'RetailDate'], inplace=True)

# Remove duplicates by orderNumber
df.drop_duplicates(subset=['orderNumber'], keep='last', inplace=True)

# Get only Finland orders
df = df[(df['RetailCountry'] == 'FIN')]

# Remove unnecessary columns
df.drop(['vehOrderClassCode', 'RetailNSC', 'NSCModelGroupCode',
       'ReportingModelGroup', 'FleetCode', 'DealerCodeRetail',
       'orderCreatedDate', 'RetailCountry', 'ModelGroup',
       'EndItem', 'EIColourCode', 'DealerDeliveryAddrNoRetail',
       'FinancialOwnerCategoryCurrent', 'FinancialOwnerCategoryNext',
       'RetailCount'
       ], axis=1, inplace=True)

In [4]:
# Convert RetailDate to date
df['Date'] = pd.to_datetime(df['RetailDate'], dayfirst=True)
df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()

In [5]:
# Generating index by date, grouping by and counting
df = df.groupby(['Date']).size().reset_index(name='Orders')

In [6]:
# Set index in both dataframes
df = df.set_index("Date")
df_ext = df_ext.set_index("Date")

# Merge both dataframes
data = pd.concat([df,df_ext], axis=1)

# Preview dataframe
data

,Orders,GDP,CCI,CPI,Unemployment_rate,Long_term_interest_rates,Avg_Fuel_Price,TIV
Date,,,,,,,,
2011-01-01,901,47117,100.73760,3.037052,8.3,3.270000,1.460000,14993
2011-02-01,671,47117,100.75380,3.329970,8.1,3.410000,1.460000,10270
2011-03-01,917,47117,100.68170,3.300893,8.9,3.450000,1.516667,13927
2011-04-01,785,49810,100.43610,3.160000,7.6,3.570000,1.526667,12491
2011-05-01,764,49810,99.90539,3.344348,7.7,3.320000,1.540000,14219
...,...,...,...,...,...,...,...,...
2022-04-01,316,68572,96.66453,5.738172,6.3,1.177000,2.216667,7220
2022-05-01,224,68572,95.99726,6.962626,6.1,1.451962,2.320000,8798
2022-06-01,231,68572,95.40240,7.792331,6.9,2.072276,2.560000,8305


In [7]:
# Save data to Excel so we don't need to run these steps every time
data.to_excel("Datasets/Dataset.xlsx") 